<a href="https://colab.research.google.com/github/cianc/AoC2023/blob/main/day10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
TEST = False #@param {type: "boolean"}
TEST_INPUT1 = 'day10_test1.txt'
TEST_INPUT2 = 'day10_test4.txt'
INPUT = 'day10_part1.txt'
PART=2 #@param {type: "integer"}


In [ ]:
import math
import time

NORTH = (0, -1)
SOUTH = (0, 1)
WEST = (-1, 0)
EAST = (1, 0)

NORTH_ALLOWED = {NORTH: ('|', '7', 'F', 'S')}
SOUTH_ALLOWED = {SOUTH: ('|', 'J', 'L', 'S')}
WEST_ALLOWED = {WEST: ('-', 'L', 'F', 'S')}
EAST_ALLOWED = {EAST: ('-', 'J', '7', 'S')}

PIPES_AND_CONNECTIONS = {'|': {**NORTH_ALLOWED, **SOUTH_ALLOWED},
         '-': {**WEST_ALLOWED, **EAST_ALLOWED},
         'L': {**NORTH_ALLOWED, **EAST_ALLOWED},
         'J': {**NORTH_ALLOWED, **WEST_ALLOWED},
         '7': {**SOUTH_ALLOWED, **WEST_ALLOWED},
         'F': {**SOUTH_ALLOWED, **EAST_ALLOWED},
         'S': {**NORTH_ALLOWED, **SOUTH_ALLOWED, **WEST_ALLOWED, **EAST_ALLOWED}}

PIPES = PIPES_AND_CONNECTIONS.keys()
TO_FROM_MAPPING = {NORTH: SOUTH, SOUTH: NORTH, EAST: WEST, WEST: EAST}

def get_input() -> list[str]:
  if PART == 1:
    input = TEST_INPUT1 if TEST else INPUT
  else:
    input = TEST_INPUT2 if TEST else INPUT

  with open(input, 'r') as f:
    rows = f.read().splitlines()
  return rows

def get_map(input: list[str]) -> list[tuple[int, int]]:
  # Need to transpose rows and columns so that the first list index is
  # the x coordinate and the second is y like god intended
  map = [[] for i in range(len(input[0]))]
  for row in input:
    for i, value in enumerate(row):
      map[i].append(value)

  return map

def find_loop(rows: list[str]) -> list[tuple[int, int]]:
  max_x_index = len(map) - 1
  max_y_index = len(map[0]) - 1
  for x, column in enumerate(map):
    for y, val in enumerate(column):
      if val == 'S':
        start_position = (x, y)
        break
    if val == 'S':
      break

  loop = [start_position]
  current_position = start_position
  current_symbol = 'S'
  from_direction = None
  while True:
    for offset in [dir for dir in (NORTH, SOUTH, EAST, WEST) if dir != from_direction]:
      new_position = (current_position[0] + offset[0], current_position[1] + offset[1])
      if new_position[0] > max_x_index or new_position[1] > max_y_index:
        continue

      new_symbol = map[new_position[0]][new_position[1]]
      if new_symbol == '.':
        continue

      # S is allowed to connect to all pipes in all direction
      if current_symbol == 'S':
        allowed_connections = PIPES
      else:
        allowed_connections = PIPES_AND_CONNECTIONS[current_symbol].get(offset, [])
      if new_symbol in allowed_connections:
        if new_symbol == 'S':
          return loop
        current_position = new_position
        current_symbol = new_symbol
        from_direction = TO_FROM_MAPPING[offset]
        loop.append(current_position)
        break

def find_enclosed_tiles(map: list[tuple[int, int]], loop: list[tuple[int, int]]) -> list[tuple[int, int]]:
  enclosed_tiles = []
  for y in range(len(map[0])):
    inside_loop = False
    prev_pipe = None
    for x in range(len(map)):
      if (x, y) not in loop:
        if inside_loop:
          enclosed_tiles.append((x,y))
        continue

      tile = map[x][y]

      # We don't care about horizontal pipes:
      if tile == '-':
        continue

      match tile:
        # A '|' or 'S' always flips us in/out of the loop
        case '|':
          inside_loop = not inside_loop

        case 'S':
          inside_loop = not inside_loop

        # '7' preceded by an 'L' forms a single logical pipe wall, so
        # don't flip after an 'L'.
        case '7':
          if prev_pipe == 'L':
            pass
          else:
            inside_loop = not inside_loop

        # 'F' has no allowed connections from the west, so always
        # acts as pipe wall
        case 'F':
          inside_loop = not inside_loop

        # 'J' preceded by an 'F' forms a single logical pipe wall, so
        # don't flip after an 'F'.
        case 'J':
          if prev_pipe == 'F':
            pass
          else:
            inside_loop = not inside_loop

        # 'L' has no allowed connections from the west, so always
        # acts as pipe wall
        case 'L':
          inside_loop = not inside_loop
      prev_pipe = tile
  return enclosed_tiles


input = get_input()
map = get_map(input)
start_time = time.time()
loop = find_loop(map)
print(f'hop count: {math.ceil(len(loop)/2)}')
enclosed_tiles = find_enclosed_tiles(map, loop)
print(f'{len(enclosed_tiles)} - enclosed_tiles: {enclosed_tiles}')
end_time = time.time()
print(f'run time: {end_time - start_time} (s)')